In [1]:
# We are using black as a code formatter
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import requests
import json
import datetime
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

In [3]:
# pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

<IPython.core.display.Javascript object>

Extraigamos los archivos parquet y los convertimos a dataframe

In [6]:
df = pd.read_parquet("../datasets/df_locationsWithAggregates.parquet")

<IPython.core.display.Javascript object>

In [7]:
df

,Id,Name,Iso2,Iso3,Longitude,Latitude,Region,SubRegion,WorldBankIncomeGroup,UNDevelopmentGroup
0,4,Afghanistan,AF,AFG,67.709953,33.939110,Asia,Southern Asia,Low-income countries,Least developed countries
1,8,Albania,AL,ALB,20.168331,41.153332,Europe,Southern Europe,Upper-middle-income countries,Developed regions
2,12,Algeria,DZ,DZA,1.659626,28.033886,Africa,Northern Africa,Lower-middle-income countries,Other developing regions
3,16,American Samoa,AS,ASM,-170.696182,-14.306021,Oceania,Polynesia,Upper-middle-income countries,Other developing regions
4,20,Andorra,AD,AND,1.521801,42.506287,Europe,Southern Europe,High-income countries,Developed regions
...,...,...,...,...,...,...,...,...,...,...
232,862,Venezuela (Bolivarian Republic of),VE,VEN,-66.589729,6.423750,Latin America and the Caribbean,South America,No income group available,Other developing regions
233,876,Wallis and Futuna Islands,WF,WLF,-178.116501,-14.293800,Oceania,Polynesia,No income group available,Other developing regions
234,882,Samoa,WS,WSM,-172.104630,-13.759029,Oceania,Polynesia,Lower-middle-income countries,Other developing regions
235,887,Yemen,YE,YEM,48.516388,15.552727,Asia,Western Asia,Low-income countries,Least developed countries


<IPython.core.display.Javascript object>

Maybe if we want to filter by WorldBankIncomeGroup (This step is not required.)

In [9]:
df_high_income = df[(df["WorldBankIncomeGroup"] == "High-income countries")]
df_low_income = df[(df["WorldBankIncomeGroup"] == "Low-income countries")]
df_upper_middle_income = df[
    (df["WorldBankIncomeGroup"] == "Upper-middle-income countries")
]
df_lower_middle_income = df[
    (df["WorldBankIncomeGroup"] == "Lower-middle-income countries")
]

<IPython.core.display.Javascript object>

In [10]:
df_high_income

,Id,Name,Iso2,Iso3,Longitude,Latitude,Region,SubRegion,WorldBankIncomeGroup,UNDevelopmentGroup
4,20,Andorra,AD,AND,1.521801,42.506287,Europe,Southern Europe,High-income countries,Developed regions
6,28,Antigua and Barbuda,AG,ATG,-61.789180,17.078670,Latin America and the Caribbean,Caribbean,High-income countries,Other developing regions
9,36,Australia,AU,AUS,133.775131,-25.274399,Oceania,Australia/New Zealand,High-income countries,Developed regions
10,40,Austria,AT,AUT,14.550072,47.516232,Europe,Western Europe,High-income countries,Developed regions
11,44,Bahamas,BS,BHS,-77.396278,25.034281,Latin America and the Caribbean,Caribbean,High-income countries,Other developing regions
...,...,...,...,...,...,...,...,...,...,...
216,796,Turks and Caicos Islands,TC,TCA,-71.797928,21.694025,Latin America and the Caribbean,Caribbean,High-income countries,Other developing regions
222,826,United Kingdom,GB,GBR,-3.435973,55.378052,Europe,Northern Europe,High-income countries,Developed regions
225,833,Isle of Man,IM,IMN,-4.548056,54.236107,Europe,Northern Europe,High-income countries,Developed regions
228,850,United States Virgin Islands,VI,VIR,-64.775520,17.740460,Latin America and the Caribbean,Caribbean,High-income countries,Other developing regions


<IPython.core.display.Javascript object>